In [ ]:
import DeepFMKit.core as dfm
import matplotlib.pyplot as plt
import numpy as np
import scipy.constants as sc

# Instantiate the main framework
dff = dfm.DeepFitFramework()

# --- 1. Define the Single, Shared Laser Source ---
laser_config = dfm.LaserConfig(label="main_laser")
laser_config.f_mod = 1000 # Modulation frequency (Hz)
laser_config.f_n = 1e6 # Laser frequency noise at 1 Hz (Hz/rtHz)
laser_config.amp_n = 1e-5 # Laser amplitude noise (1/rtHz)

# --- 2. Define the Main Interferometer ---
main_ifo_config = dfm.InterferometerConfig(label="dynamic_ifo")
main_ifo_config.ref_arml = 0.1 # Reference arm length (m)
main_ifo_config.meas_arml = 0.3 # Measurement arm length (m)
main_ifo_config.arml_mod_f = 1.0 # Measurement arm modulation frequency (Hz)
main_ifo_config.arml_mod_amp = 0. # Armlength modulation amplitude (m)
main_ifo_config.arml_mod_n = 0. # Armlength modulation amplitude noise (m/rtHz)

# --- 3. Set Modulation Depth by Adjusting Laser's `df` ---
m_target = 6.0 # Target effective modulation index (rad)
opd = main_ifo_config.meas_arml - main_ifo_config.ref_arml # Optical pathlength difference (m)
df_required = (m_target * sc.c) / (2 * np.pi * opd) # Required laser modulation amplitude (Hz)
laser_config.df = df_required

# --- 4. Compose the Main Channel ---
main_label = "dynamic_channel"
main_channel = dfm.DFMIObject(
    label=main_label,
    laser_config=laser_config,
    ifo_config=main_ifo_config,
    f_samp=int(200e3) # Sampling frequency (Hz)
)
dff.sims[main_label] = main_channel

# --- 6. Simulate ---
dff.simulate(
    main_label=main_label,
    n_seconds=2 # Simulation length in seconds
)

# NLS fit
dff.fit(label=main_label, fit_label="nls", n=20)

In [ ]:
dff.raws['dynamic_channel'].sim.laser

In [ ]:
# Run the EKF with default tuning parameters
dff.fit_ekf(label=main_label, fit_label="ekf")

# Run with custom tuning
custom_Q = [1e-9, 1e-9, 1e-7, 1e-7, 1e-9]
custom_R = 0.001
dff.fit_ekf(label=main_label, fit_label="ekf_tuned", Q_diag=custom_Q, R_val=custom_R)

In [ ]:
ax = dff.plot(labels=['nls', 'ekf', 'ekf_tuned'])
ax[0].legend()
plt.show()